In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

In [2]:
OutPath = 'OUTPUT/filtered-companies.csv'

In [3]:
dbName="companies"
mongodbURL = f"mongodb://localhost/{dbName}"

# access database and avoid timeouts
mclient = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = mclient.get_database()

In [4]:
cur = db.list_collections()

In [5]:
for col in cur:
    print(col)

{'name': 'companies', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('76992dc7-b5e1-4dcd-9a37-b36c0e165615')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'companies.companies'}}


In [6]:
# query to only bring certain attributes from the complete collection
# relevant data:
# - offices
# - name
# - category_code
# - founded_year
# - succesful startups that raised more then 1M USD or 750K EUR

all_offices = list(
    db.companies.find({
        
    },
      {"offices":1,"name":1,"category_code":1, "founded_year":1,
      "acquisition":1}))

In [9]:
from pandas.io.json import json_normalize
pd.DataFrame(json_normalize(all_offices)).head(3)

/home/rh/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,_id,name,category_code,founded_year,acquisition,offices,acquisition.price_amount,acquisition.price_currency_code,acquisition.term_code,acquisition.source_url,acquisition.source_description,acquisition.acquired_year,acquisition.acquired_month,acquisition.acquired_day,acquisition.acquiring_company.name,acquisition.acquiring_company.permalink
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,1996.0,NaN,"[{'description': 'Headquarters', 'address1': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,2005.0,NaN,"[{'description': '', 'address1': '710 - 2nd Av...",30000000.0,USD,cash_and_stock,http://allthingsd.com/20131216/viggle-tries-to...,Viggle Tries to Bulk Up Its Social TV Busines...,2013.0,12.0,16.0,Viggle,viggle
2,52cdef7c4bab8bd675297d8c,Zoho,software,2005.0,NaN,"[{'description': 'Headquarters', 'address1': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Use .explode() to turn "offices" column, which is a series with a list of dictionaries,
# into to multiple rows.
# Each row represents an office location

companydata = pd.DataFrame(all_offices).explode("offices")
companydata.head(3)

,_id,name,category_code,founded_year,acquisition,offices
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,1996.0,None,"{'description': 'Headquarters', 'address1': '4..."
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,2005.0,"{'price_amount': 30000000, 'price_currency_cod...","{'description': '', 'address1': '710 - 2nd Ave..."
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,2005.0,"{'price_amount': 30000000, 'price_currency_cod...","{'description': '', 'address1': '270 Lafayette..."


In [13]:
companydata.to_csv(OutPath)


In [14]:
pd.json_normalize(OutPath)


AttributeError: 'str' object has no attribute 'values'